# 2. Earth Engine Basics

Module 1 is designed to give you basic skills to be able to find datasets you need for your project, filter them to your region of interest, apply basic processing and export the results. Mastering this will allow you to start using Earth Engine for your project quickly and save a lot of time pre-processing the data.

In [28]:
from IPython.display import YouTubeVideo

YouTubeVideo(id="kpfncBHZBto", width=600, height=300)

In [1]:
%load_ext autoreload
%load_ext dotenv
%autoreload 2
%dotenv

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import geemap
from geemap import Map

In [3]:
import ee
ee.Initialize()

## 01. Hello World

Let's familiarize ourselves with the basic syntax of Python + GEE:

In [4]:
print('Hello World')

Hello World


In [5]:
# Variables
city = ee.String("Bengaluru")
country = ee.String("India")
print(city, country)

ee.String({
  "constantValue": "Bengaluru"
}) ee.String({
  "constantValue": "India"
})


In [6]:
population = ee.Number(8400000)
print(population)

ee.Number({
  "constantValue": 8400000
})


In [7]:
# List
major_cities = ee.List(["Mumbai", "Delhi", "Chennai", "Kolkata"])
print(major_cities)

ee.List({
  "constantValue": [
    "Mumbai",
    "Delhi",
    "Chennai",
    "Kolkata"
  ]
})


In [8]:
# Dictionary
city_data = ee.Dictionary({
    "city": city,
    "population": population,
    "elevation": 930
})
print(city_data)

ee.Dictionary({
  "constantValue": {
    "city": "Bengaluru",
    "elevation": 930,
    "population": 8400000
  }
})


In [9]:
# Function
greet = lambda name: f"Hello {name}"
print(greet("World"))

# This is a comment

Hello World


### Exercise

In [10]:
# These are the 5 largest cities in the world: 
# Tokyo, Delhi, Shanghai, Mexico City, Sao Paulo
# Create a list named 'largeCities'
large_cities = ee.List(["Tokyo", "Delhi", "Shanghai", "Mexico City", "Sao Paulo"])

# The list should have names of all the above cities
# Print the list 
print(large_cities, large_cities.getInfo())

ee.List({
  "constantValue": [
    "Tokyo",
    "Delhi",
    "Shanghai",
    "Mexico City",
    "Sao Paulo"
  ]
}) ['Tokyo', 'Delhi', 'Shanghai', 'Mexico City', 'Sao Paulo']


## 02. Working with Image Collections

Most datasets in Earth Engine come as an `ImageCollection`. An `ImageCollection` is a dataset that consists of images taken at different times and locations, usually from the same satellite or data provider. 

You can load a collection by searching the Earth Engine Data Catalog for the `ImageCollection` ID or from the map UI.

Search for the Sentinel-2 Level 1C dataset and you will find its id `COPERNICUS/S2_SR`.

In [14]:
# Init map to look for datasets
m = Map()
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Copy the code snippet associated with the data production and visualize the collection:

In [15]:
def maskS2clouds(img):
    """Masks clouds using the Sentinel-2 QA bands.
    
    # Params
        img (ee.Image).
    
    # Returns
        masked_img (ee.Image).
    """
    qa = img.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return img.updateMask(mask).divide(10000)


# Mask 1 year of Sentinel-2 images.
imgs = ee.ImageCollection("COPERNICUS/S2")\
         .filterDate("2018-01-01", "2018-06-30")\
         .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))\
         .map(maskS2clouds)

# Set visualization parameters
viz = {
  "min": 0.0,
  "max": 0.3,
  "bands": ['B4', 'B3', 'B2'],
}

m.setCenter(-5.436518, 35.5850477, 12)
m.addLayer(imgs.median(), viz, "RGB")

In the code snippet, You will see a function `m.setCenter()` which sets the viewport to a specific location and zoom level. The function takes the `X` coordinate (longitude), `Y` coordinate (latitude) and Zoom Level parameters. 

Replace the `X` and `Y` coordinates with the coordinates of your city and click Run to see the images of your city.

### Exercise

Find the 'Sentinel-2 Level-2A' dataset.
Copy/paste the code snippet

Change the code to display images for your home city:

In [16]:
dataset = ee.ImageCollection('COPERNICUS/S2_SR')\
             .filterDate('2020-01-01', '2020-06-30')\
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))\
             .map(maskS2clouds)

# Set visualization parameters
viz = {
  "min": 0.0,
  "max": 0.3,
  "bands": ['B4', 'B3', 'B2'],
}

# Set center on my city and visualize
m.setCenter(-5.436518, 35.5850477, 12)
m.addLayer(dataset.mean(), viz, "L2-A RGB")

## 03. Filtering Image Collections

The original image collection contains all imagery ever collected by teh sensor. The entire collection is not very useful. Most applications require a subset of the images.

We use filters to select the appropriate images. There are many types of filter functions. The `ee.Filter.*` modules contains all available filters. Select a filter then run the `filter()` function with its parameters.

We will learn about 3 main types of filtering techniques:

- Filter by metadata: uses filters such as `ee.Filter.eq()`, `ee.Filter.lt()`, etc. 
    - You can filter by PATH/ROW values, Orbit number, Cloud cover etc.
- Filter by date: You can select images in a particular date range using filters such as `ee.Filter.date()`.
- Filter by location: You can select the subset of images with a bounding box, location or geometry using the `ee.Filter.bounds()`. 
    - You can also use the drawing tools to draw a geometry for filtering.

After applying the filters, you can use the size() function to check how many images match the filters.

In [17]:
# Create a point of interest
geometry = ee.Geometry.Point([77.60412933051538, 12.952912912328241])

# Set the map center
m.centerObject(geometry, 10)

# Get the image collection
s2 = ee.ImageCollection("COPERNICUS/S2")

# Filter by metadata
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))

# Filter by date
filtered = filtered.filter(ee.Filter.date('2019-01-01', '2020-01-01'))

# Filter by location
filtered = filtered.filter(ee.Filter.geometry(geometry))

# Instead of applying filters one after the other, we can "chain" them
# Use the `.` notation to apply all the filters together
filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
             .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
             .filter(ee.Filter.geometry(geometry))

# Print how many images we have after filtering
print(filtered.size().getInfo())

# Visualize
viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'],
}

m.addLayer(filtered, viz, 'Filtered') 

30


### Exercise

Add one more filter in the script below to select images from only one of the satellites: **Sentinel-2A**, from the Sentinel-2 constellation

- Hint1: Use the 'SPACECRAFT_NAME' property
- Hint2: Use the ee.Filter.eq() filter

In [18]:
geometry = ee.Geometry.Point([77.60412933051538, 12.952912912328241])

m.centerObject(geometry, 10)

imgs = ee.ImageCollection("COPERNICUS/S2")\
         .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 30))\
         .filter(ee.Filter.date("2019-01-01", "2020-01-01"))\
         .filter(ee.Filter.geometry(geometry))\
         .filter(ee.Filter.eq("SPACECRAFT_NAME", "Sentinel-2A"))

print(imgs.size().getInfo())

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'],
}

m.addLayer(imgs.median(), viz, "Filtered")

16


## 04. Creating Mosaics and Composites from `ImageCollections`

The default order of the collection is by **date**. So when you display the collection, it implicitly creates a mosaic with the latest pixels on top. You can call `.mosaic()` on an `ImageCollection` to create a mosaic image from the pixels at the top.

We can also create a composite image by applying selection criteria to each pixel from all pixels in the stack. Here we use the `median()` function to create a composite where each pixel value is the median of all pixels from the stack.

Tip: If you need to create a mosaic where the images are in a specific order, you can use the `.sort()` function to sort your collection by a property first.

In [19]:
geometry = ee.Geometry.Point([77.60412933051538, 12.952912912328241])
s2 = ee.ImageCollection("COPERNICUS/S2")

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'],
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.geometry(geometry))

# Create the mosaic
mosaic = filtered.mosaic()
 
# Create the median composite
median_composite = filtered.median()

# Visualize
m.addLayer(filtered, viz, 'Filtered Collection')
m.addLayer(mosaic, viz, 'Mosaic')
m.addLayer(median_composite, viz, 'Median Composite')

### Exercise

Create a median composite for the year 2020 and load it to the map.

Compare both the composites to see the changes in your city.

In [17]:
geometry = ee.Geometry.Point([-5.436518, 35.5850477])
s2 = ee.ImageCollection("COPERNICUS/S2")

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'],
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2020-01-01', '2021-01-01'))\
    .filter(ee.Filter.geometry(geometry))

# Create the mosaic
mosaic = filtered.mosaic()
 
# Create the median composite
median_composite = filtered.median()

# Visualize
m.setCenter(-5.436518, 35.5850477, 12)
m.addLayer(mosaic, viz, 'Mosaic')
m.addLayer(median_composite, viz, 'Median Composite')

## 05. Working with Feature Collections

Feature Collections are similar to Image Collections, but they contain Features, not images. They are equivalent to Vector Layers in a GIS. We can load, filter and display Feature Collections using similar techniques that we have learned so far.

Search for GAUL Second Level Administrative Boundaries and load the collection. This is a global collection that contains all Admin2 boundaries. We can apply a filter using the ADM1_NAME property to get all Admin2 boundaries (i.e. Districts) from a state.

In [20]:
# Get administrative boundaries
admins = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")

# Filter 
karnataka = admins.filter(ee.Filter.eq('ADM1_NAME', 'Karnataka'))

viz = {'color': 'red'}
m.centerObject(karnataka)
m.addLayer(karnataka, viz, 'Karnataka Districts')

### Exercise

In [22]:
# Apply a filter to select only the 'Bangalore Urban' district
# Display only the selected district
# Hint: The district names are in ADM2_NAME property
admins = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
bangalore = admins.filter(ee.Filter.eq('ADM1_NAME', 'Karnataka')).filter(ee.Filter.eq("ADM2_NAME", "Bangalore Urban"))

viz = {"color": "red"}
m.centerObject(bangalore)
m.addLayer(bangalore, viz, 'bangalore')

## 06. Importing Data

You can import vector or raster data into Earth Engine. We will now import a shapefile of Urban Areas for Natural Earth. 

Unzip the `ne_10m_urban_areas.zip` into a folder on your computer.

Let's import it to GEE:

In [23]:
# Upload
shp_file_path = "static/features/ne_10m_urban_areas.shp"
ee_object = geemap.shp_to_ee(shp_file_path)

### Exercise

In [25]:
# Apply a filter to select only large urban areas
# Use the property 'area_sqkm' and select all features 
# with area > 400 sq.km
large_regions = ee_object.filter(ee.Filter.gt("area_sqkm", 400))

## 07. Clipping Images

It is often desirable to clip the images to your area of interest. You can use the clip() function to mask out an image using a geometry.

In [26]:
s2 = ee.ImageCollection("COPERNICUS/S2")
urban = ee.FeatureCollection("users/ujavalgandhi/e2e/ne_10m_urban_areas")

# Find the feature id by adding the layer to the map and using Inspector.
filtered = urban.filter(ee.Filter.eq('system:index', '00000000000000002bf8'))

geometry = filtered.geometry()

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'], 
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.geometry(geometry))

image = filtered.median() 

clipped = image.clip(geometry)

m.centerObject(clipped)
m.addLayer(clipped, viz, 'Clipped')

### Exercise

- Add the imported table to the Map
- Use the Inspector to find the id of your home city or any urban area of your choice
- Change the filter to use the id of the selected feature

In [27]:
urban = ee.FeatureCollection("users/ujavalgandhi/e2e/ne_10m_urban_areas")

# Find the feature id by adding the layer to the map and using Inspector.
filtered = urban.filter(ee.Filter.eq('area_sqkm', 1116.547))

m.addLayer(urban, {}, "rois")

geometry = filtered.geometry()

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'], 
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.geometry(geometry))

image = filtered.median() 

clipped = image.clip(geometry)

m.addLayer(clipped, viz, 'Clipped')

## 08. Exporting Data

Earth Engine allows for exporting both vector and raster data to be used in an external program. Vector data can be exported as a CSV or a Shapefile, while Rasters can be exported as GeoTIFF files. We will now export the Sentinel-2 Composite as a GeoTIFF file.

In [27]:
s2 = ee.ImageCollection("COPERNICUS/S2")
urban = ee.FeatureCollection("users/ujavalgandhi/e2e/ne_10m_urban_areas")
filtered = urban.filter(ee.Filter.eq('area_sqkm', 1116.547))

geometry = filtered.geometry()

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'], 
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.geometry(geometry))

image = filtered.median()

clipped = image.clip(geometry)

m.addLayer(clipped, viz, 'Clipped') 

exportImage = clipped.select('B.*')

# Export
geemap.ee_export_image(exportImage, filename="static/imgs/img.tif", scale=250, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /Users/akramzaytar/projects/Learn/GEE/ezmap/notebooks/static/imgs/img.tif


In [69]:
# Rather than exporting raw bands, we can apply a rendered image
# visualize() function allows you to apply the same parameters 
# that are used in earth engine which exports a 3-band RGB image
visualized = clipped.visualize(**viz)

# Now the 'visualized' image is RGB image, no need to give visParams
m.addLayer(visualized, {}, 'Clipped')

geemap.ee_export_image(visualized, filename="static/imgs/viz_img.tif", scale=250, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /Users/akramzaytar/projects/Learn/GEE/ezmap/notebooks/static/imgs/viz_img.tif


### Exercise

Write the export function to export the results for your chosen urban area:

In [70]:
urban = ee.FeatureCollection("users/ujavalgandhi/e2e/ne_10m_urban_areas")

# Find the feature id by adding the layer to the map and using Inspector.
filtered = urban.filter(ee.Filter.eq('area_sqkm', 1116.547))

m.addLayer(urban, {}, "rois")

geometry = filtered.geometry()

viz = {
  "min": 0.0,
  "max": 3000,
  "bands": ['B4', 'B3', 'B2'],
}

filtered = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))\
    .filter(ee.Filter.date('2019-01-01', '2020-01-01'))\
    .filter(ee.Filter.geometry(geometry))

image = filtered.median() 

clipped = image.clip(geometry)

geemap.ee_export_image(clipped, filename="static/imgs/my_viz.tif", scale=250, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /Users/akramzaytar/projects/Learn/GEE/ezmap/notebooks/static/imgs/my_viz.tif


# Assignment 1

Export the Night Lights images for May,2015 and May,2020

Workflow:
1. [x] Load the VIIRS Nighttime Day/Night Band Composites collection
2. [x] Filter the collection to the date range
3. [x] Extract the 'avg_rad' band which represents the nighttime lights
4. [x] Clip the image to the geometry of your city
5. [x] Export the resulting image as a GeoTIFF file.

In [76]:
admins = ee.FeatureCollection('FAO/GAUL/2015/level2')
tetouan = admins.filter(ee.Filter.eq("ADM2_CODE", 21817))
m.addLayer(tetouan, {}, "FAO/GAUL/2015/level2")

In [80]:
# Load
lights = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')\
    .filter(ee.Filter.date("2020-05-01", "2020-06-01"))\
    .select(["avg_rad"]).first()

tetouan_viz = lights.clip(tetouan.geometry())

# Visualize
m.addLayer(tetouan_viz, {}, "Lights_2")

In [81]:
geemap.ee_export_image(tetouan_viz, filename="static/imgs/my_lights.tif", scale=250, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /Users/akramzaytar/projects/Learn/GEE/ezmap/notebooks/static/imgs/my_lights.tif


In [ ]:
geemap.ee_search()

---